## 爬取 PyODPS[latest] 并转换为 PDF
- 爬取主链接
- 根据主链接爬取子连接
- 参考子链接爬取HTML并转换为PDF
- 将所有 PDF 整合为一个PDF
---
- 注 :
    - PyOdps PDF在线最新版本
        - 0.3.12

In [15]:
import re
import pdfkit
import pandas as pd
from urllib import urlopen
from bs4 import BeautifulSoup

# 设置 pandas 显示参数
pd.set_option('display.width',200)
pd.set_option('display.max_rows',1000)
pd.set_option('display.max_columns',50)
pd.set_option('display.max_colwidth',500)

### 爬取主链接
#### 爬取PyODPS Docs主页面

In [9]:
url='http://pyodps.readthedocs.io/zh_CN/latest/index.html'
html=urlopen(url).read().decode('utf8')
soup=BeautifulSoup(html,'lxml')

#### 取值最新文档首页 API及标题

In [10]:
# 主链接 (API)
api=soup.find(name='link', attrs={'rel':'canonical'}).get('href')
# 获取文档标题
title=soup.find('link',attrs={"href":"#","rel":"top"}).get('title').replace(' ','_')

# 获取首页超链接 (href)
hrefs=[]
div_s=soup.find_all(name='div',attrs={'aria-label':'main navigation','role':'navigation'})[0]
for tag_a in div_s.find_all(name='a',attrs={'class':'reference internal'}):
    content_name=tag_a.get_text()
    url=api+tag_a.get('href')
    hrefs.append([content_name,url])

#### 美化 DataFrame 显示效果函数

In [20]:
'''
设置悬停效果
'''
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])
'''
美化DataFrame显示效果
'''
def display_prettify(df):
    from IPython.display import HTML

    styles = [
        hover(),
        dict(selector="th", props=[("font-size", "100%"),
                                   ("text-align", "center")]),
        dict(selector="td", props=[("text-align", "left")]),
        dict(selector="caption", props=[("caption-side", "left")])
    ]
    return df.style.set_table_styles(styles).set_caption("Hover to highlight.")

#### 首页超连接(href)打印显示

In [13]:
df=pd.DataFrame(hrefs, columns=['content_name','href'])

display_prettify(df)

,content_name,href
0,安装指南,http://pyodps.readthedocs.io/zh_CN/latest/installation-pub-zh.html
1,基本操作,http://pyodps.readthedocs.io/zh_CN/latest/base-zh.html
2,DataFrame,http://pyodps.readthedocs.io/zh_CN/latest/df-zh.html
3,机器学习,http://pyodps.readthedocs.io/zh_CN/latest/ml-zh.html
4,交互体验增强,http://pyodps.readthedocs.io/zh_CN/latest/interactive-zh.html
5,配置选项,http://pyodps.readthedocs.io/zh_CN/latest/options-zh.html
6,API Reference,http://pyodps.readthedocs.io/zh_CN/latest/api.html


### 根据主链接爬取子连接

In [ ]:
hrefs_2=[] # 有序列表，存储主、子链接并与文档目录层次结构保持一致性

for name,url in hrefs:
    if url not in [hf[1] for hf in hrefs_2]: # href 不在 hrefs_2中，则追加
        hrefs_2.append([name,url])
    t_html=urlopen(url).read().decode('utf8')
    
    # 根据正则表达式 查找当前目录主题
    f_re='<a class="current reference internal".*?</a><ul>(.*?)</ul>'
    if len(re.findall(f_re, t_html, re.I|re.S|re.M)) !=0 :
        target_s = re.findall(f_re, t_html, re.I|re.S|re.M)[0]

        # 根据正则表达式 获取当前子主题链接
        t_re='<a class="reference internal" href="(.*?)">(.*?)</a>'
        for href,name in re.findall(t_re, target_s, re.I|re.S|re.M):
            if href.strip().endswith('.html'):
                hrefs_2.append([name,api+href])

In [22]:
display_prettify(pd.DataFrame(hrefs_2))

,0,1
0,安装指南,http://pyodps.readthedocs.io/zh_CN/latest/installation-pub-zh.html
1,基本操作,http://pyodps.readthedocs.io/zh_CN/latest/base-zh.html


#### 显示PyODPS 所有链接

In [105]:
pd.DataFrame(hrefs_2)

,0,1
0,安装指南,http://pyodps.readthedocs.io/zh_CN/latest/installation-pub-zh.html
1,基本操作,http://pyodps.readthedocs.io/zh_CN/latest/base-zh.html
2,项目空间,http://pyodps.readthedocs.io/zh_CN/latest/base-projects-zh.html
3,表,http://pyodps.readthedocs.io/zh_CN/latest/base-tables-zh.html
4,SQL,http://pyodps.readthedocs.io/zh_CN/latest/base-sql-zh.html
5,任务实例,http://pyodps.readthedocs.io/zh_CN/latest/base-instances-zh.html
6,资源,http://pyodps.readthedocs.io/zh_CN/latest/base-resources-zh.html
7,函数,http://pyodps.readthedocs.io/zh_CN/latest/base-functions-zh.html
8,模型,http://pyodps.readthedocs.io/zh_CN/latest/base-models-zh.html
9,DataFrame,http://pyodps.readthedocs.io/zh_CN/latest/df-zh.html


### 参考子链接爬取HTML并转换为PDF

In [24]:
for name,href in hrefs_2:
    pdfkit.from_url(href,'./tmp/'+name+'.pdf')

Loading pages (1/6)
QFont::setPixelSize: Pixel size <= 0 (0)                     ] 35%
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      
Loading pages (1/6)
QFont::setPixelSize: Pixel size <= 0 (0)                     ] 26%
Counting pages (2/6)                                               
Resolving links (4/6)                                                       
Loading headers and footers (5/6)                                           
Printing pages (6/6)
Done                                                                      


### 整合所有 PDF

In [114]:
from PyPDF2 import PdfFileMerger

# 创建 PdfFileMerger 对象，合并PDFs
merger = PdfFileMerger()
for name, url in hrefs_2:
    t_input = open('./tmp/'+name+'.pdf', 'rb')
    merger.append(t_input)

# 流输出
output = open(title+".pdf", "wb")
merger.write(output)

# 关闭文件流
output.close()
merger.close()